In [3]:
import pandas as pd

In [6]:
raw_states_df = pd.read_csv('../data/states.csv', skiprows=range(4))

# Filters
raw_states_df.dropna(subset=['GeoName', 'Description'], inplace=True)
raw_states_df = raw_states_df[raw_states_df.GeoName != 'United States']

# Drop Columns
raw_states_df.drop(['GeoFips', 'LineCode'], axis=1, inplace=True)

# Renaming
raw_states_df.rename(columns={'GeoName': 'state'}, inplace=True)
raw_states_df.Description.replace({
    'Personal income (millions of dollars)': 'personal_income',
    'Population (persons) 1/': 'population',
    'Per capita personal income (dollars) 2/': 'pipc'
}, inplace=True)

raw_states_df.head()

,state,Description,1929,1930,1931,1932,1933,1934,1935,1936,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
3,Alabama,personal_income,843.2,697.5,583.7,421.9,435.6,556.3,586.9,683.2,...,173361.7,175185.3,181078.8,189115.4,192363.5,199336.4,207053.6,215930.4,228748.8,244975.8
4,Alabama,population,2644000,2647000,2649000,2653000,2661000,2685000,2719000,2743000,...,4839261.0,4864399.0,4886793.0,4908162.0,4930595.0,4952202.0,4976395.0,5003418.0,5024803.0,5039877.0
5,Alabama,pipc,319,263,220,159,164,207,216,249,...,35824.0,36014.0,37055.0,38531.0,39014.0,40252.0,41607.0,43157.0,45524.0,48608.0
6,Alaska *,personal_income,(NA),(NA),(NA),(NA),(NA),(NA),(NA),(NA),...,39266.0,38978.0,41210.0,42555.1,41837.4,42497.1,44102.7,45293.9,46430.3,49190.3
7,Alaska *,population,(NA),(NA),(NA),(NA),(NA),(NA),(NA),(NA),...,731106.0,738057.0,737638.0,739127.0,743410.0,741949.0,737717.0,734823.0,732441.0,732673.0


In [8]:
# Pivot
pivoted_states_df = raw_states_df.set_index(['state', 'Description']).stack().unstack(1)
pivoted_states_df.reset_index(inplace=True)

# Renaming
pivoted_states_df.rename(columns={'level_1': 'year'}, inplace=True)

# Datatypes
pivoted_states_df.year = pd.to_numeric(pivoted_states_df.year, errors='coerce', downcast='integer')
pivoted_states_df.personal_income = pd.to_numeric(pivoted_states_df.personal_income, errors='coerce', downcast='float')
pivoted_states_df.pipc = pd.to_numeric(pivoted_states_df.pipc, errors='coerce', downcast='float')
pivoted_states_df.population = pd.to_numeric(pivoted_states_df.population, errors='coerce', downcast='integer')

# Cleaning
pivoted_states_df.state.replace({'Alaska *': 'Alaska'}, inplace=True)

pivoted_states_df.head()

Description,state,year,personal_income,pipc,population
0,Alabama,1929,843.200012,319.0,2644000.0
1,Alabama,1930,697.500000,263.0,2647000.0
2,Alabama,1931,583.700012,220.0,2649000.0
3,Alabama,1932,421.899994,159.0,2653000.0
4,Alabama,1933,435.600006,164.0,2661000.0


In [9]:
pivoted_states_df.state.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Far West', 'Florida', 'Georgia', 'Great Lakes', 'Hawaii *',
       'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
       'Mideast', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New England', 'New Hampshire', 'New Jersey',
       'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
       'Oklahoma', 'Oregon', 'Pennsylvania', 'Plains', 'Rhode Island',
       'Rocky Mountain', 'South Carolina', 'South Dakota', 'Southeast',
       'Southwest', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
       'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [11]:
pivoted_states_df.dtypes

Description
state               object
year                 int16
personal_income    float32
pipc               float32
population         float64
dtype: object

In [52]:
from nba_api.stats.static import teams

In [54]:
available_teams = teams.get_teams()

In [56]:
team_states = [t['state'] for t in available_teams]

In [57]:
raw_states = pivoted_states_df.state.unique()

In [58]:
[x for x in team_states if x not in raw_states]

['Atlanta', 'Ontario']

Hay buen match entre ambas fuentes de datos. Los únicos dos casos en los que no hay match son:
1. Ontario porque no es un estados de la unión americana
2. Altanta porque no es un estado, hay que modificarlo en el proceso de limpieza de la tabla de equipos.